In [ ]:
import socket
import threading
import cv2
import numpy as np
import pickle
import sys

In [ ]:
#Creating Socket Program
protocol = socket.SOCK_STREAM
net_family = socket.AF_INET
s = socket.socket(net_family, protocol)

In [ ]:
#Binding Socket
serverip = ""
port = 8889
s.bind((serverip,port))
s.listen()

In [ ]:
#Starting Session
csession, addr = s.accept()
csession.send(b"Im server")

In [ ]:
#Selecting the Camera Input
cap = cv2.VideoCapture(0)

In [ ]:
cap.isOpened()

In [ ]:
#Function to Send the image stream to client
def Send(csession):
    while True:
        status, photo = cap.read()
        if type(photo) is type(None):
            pass
        else:
            #cv2.imshow("Video on Server", photo)
            #if cv2.waitKey(10) == 13:
            #    cv2.destroyAllWindows()
            #    break
            dumped_photo = pickle.dumps(photo)
            csession.sendall(dumped_photo)
    cap.release()

"""

def Send(csession):
    while True:
        
        status, photo = cap.read()
        if type(photo) is type(None):
            pass
        else:
            #photo_buffer = cv2.imencode('.jpg',photo)[1]
            photo_buffer = cv2.imencode('.jpg',photo)[1].tobytes()
            #photo_to_bytes = pickle.dumps(photo_buffer)
            #csession.sendall(photo_to_bytes)
            csession.sendall(photo_buffer)
    cap.release()

"""


In [ ]:
#Function to Recieve the image Stream from server 
def Receive(csession):
    while True:
        data = csession.recv(921797)
        print(sys.getsizeof(data))
        if sys.getsizeof(data) == 921797:
            photo = pickle.loads(data)
            cv2.imshow("Video from Server", photo)
            if cv2.waitKey(100) == 13:
                cv2.destroyAllWindows()
                break
        else:
            pass



"""
def Receive(csession):
    while True:
        data = csession.recv(1000000)
        encoded_photo = np.frombuffer(data , dtype = np.uint8)
        photo = cv2.imdecode(encoded_photo, cv2.IMREAD_COLOR)
        if type(photo) is type(None):
            pass
        else:
            cv2.imshow("Video from Server", photo)
            if cv2.waitKey(300) == 13:
                cv2.destroyAllWindows()
                break
    cap.release()
    
"""
"""
def Receive(csession):    
    data = b""
    payload_size = struct.calcsize("Q")

    while True:
        while len(data) < payload_size:
            packet = csession.recv(4*1024)
            if not packet: break
            data += packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q", packed_msg_size)[0]
        while len(data) < msg_size:
            data += csession.recv(4*1024)
        photo_data = data[:msg_size]
        data = data[msg_size:]
        photo = pickle.loads(photo_data)
        cv2.imshow("Video from Client", photo)
        if cv2.waitKey(200) == 13:
            cv2.destroyAllWindows()
            break

"""


In [ ]:
t1 = threading.Thread(target=Send, args=(csession,))
t2 = threading.Thread(target=Receive, args=(csession,))

In [ ]:
t1.start()
t2.start()